In [1]:
# binary classification for automl off-target
# some of TPOT pre-processing and run code from grad student Luis
## Import Libraries
# General system libraries
import os
import numpy as np
import pandas as pd
from time import time
from IPython.display import Image

# DNA/RNA Analysis Libraries (Biopython, ViennaRNA, pysster) 
# Biopython Lib
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_rna, generic_dna, generic_protein, IUPAC
# ViennaRNA Lib
import RNA
# pysster Lib
from pysster import utils
from pysster.Data import Data
from pysster.Grid_Search import Grid_Search
from pysster.One_Hot_Encoder import One_Hot_Encoder
from pysster.Alphabet_Encoder import Alphabet_Encoder

# Import TPOT libs
from tpot import TPOTClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.metrics import median_absolute_error, r2_score

# Math & Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Warnings
import warnings

Using TensorFlow backend.


In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

def confusion_plot(y_true_labels, y_pred_labels, title, class_names): 

    def plot_confusion_matrix(cm, classes,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("All data confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        print(cm)

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=False,
                          title=title)

    plt.show()
    
    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title=title)

    plt.show()

In [3]:
data_folder = "data/"
data_filename = "processed_binary_data.csv"
data_path = data_folder + data_filename
data = pd.read_csv(data_path)

#Show dataframe
data.head()

,Unnamed: 0,On-Target Site,Off-Target Site,Score,Encoded Seqs
0,0,AAATGAGAAGAAGAGGCACAGGG,AAAAGAGAAGCTGAGGCACAGGG,1.0,KKKREKEKKEZREKEEPKPKEEE
1,1,AAATGAGAAGAAGAGGCACAGGG,AAACGAGAAGAAGAGGCATAGGG,1.0,KKKMEKEKKEKKEKEEPKMKEEE
2,2,AAATGAGAAGAAGAGGCACAGGG,AAACGAGAAGAAGCTGCACATGG,1.0,KKKMEKEKKEKKEZJEPKPKJEE
3,3,AAATGAGAAGAAGAGGCACAGGG,AAAGGAGAAGGAGAGGCAGATGG,1.0,KKKJEKEKKEYKEKEEPKHKJEE
4,4,AAATGAGAAGAAGAGGCACAGGG,AAATCATATGAAGAGGCACAAGG,1.0,KKKLHKJKREKKEKEEPKPKYEE


In [4]:
# input is the newly encoded alphabet
# output is the binary score
df_data_input = data['Encoded Seqs']
df_data_output = data['Score']

print('Total number samples:', len(df_data_input))
print('Example encoding:', df_data_input[100])

Total number samples: 409002
Example encoding: EKEMPMEKEPKEKKEYKEKKEEE


In [5]:
# one-hot encode input sequences in new alphabet space 
new_alph_map = {'AA': 'K', 'AT': 'R', 'AC': 'Z', 'AG': 'Y', 'TT': 'L', 'TC': 'M', 'TG':'J', 
                'CC': 'P', 'CG':'H', 'GG': 'E'}
new_alph = list(new_alph_map.values())
#modified code from Luis to get correct format for TPOT w/ my alphabet
one_data_input = []
one = One_Hot_Encoder(''.join(new_alph))
for index, seq in df_data_input.items():
    one_hot_seq = one.encode(seq)                           #ONE HOT ENCODE, and then
    one_data_input.append(np.argmax((one_hot_seq), axis=1)) #To numerical for TPOT

# # one-hot encode
# def _get_one_hot_encoding(seq):
#   """Generate one-hot encoding for a single nucleotide sequence."""
#   return pd.get_dummies(
#       list(seq) + new_alph)[:-len(new_alph)].as_matrix().transpose()
# # now convert the data into one_hot_encoding 
# input_col_name = 'Encoded Seqs'
# one_data_input = np.stack(
#     [_get_one_hot_encoding(s) for s in df_data_input]).astype(np.float32)

In [6]:
# understand output scores
print('Number negative, positive:', sum(df_data_output)) # can just sum b/c 1s are positive (else are 0, no effect on sum)
print('Num negative:', len(df_data_output) - sum(df_data_output))
# make sure outputs are one-hot encoded
from keras.utils import to_categorical
norm_data_output = np.array(df_data_output.astype(int))


Number negative, positive: 742.0
Num negative: 408260.0


In [7]:
# TPOT code modified from Luis

# Divide dataset into training and testing sets
data_input = np.array(one_data_input) 
data_output = norm_data_output
print('Dataset has input shape:'+ str(np.shape(data_input)) + ', and output shape: '+ str(np.shape(data_output)))

# Define Train and Test ratios
train_size = 0.75
test_size = 0.25
X_train, X_test, y_train, y_test = train_test_split(data_input, data_output,train_size=train_size, test_size=test_size)
print('Data split into Train(' + str(train_size*100) + '%) and Test(' + str(test_size*100) +'%) sets')   

Dataset has input shape:(409002, 23), and output shape: (409002,)
Data split into Train(75.0%) and Test(25.0%) sets


In [8]:
# Train TPOT classifier
# Ref: https://epistasislab.github.io/tpot/api/
tpot = TPOTClassifier(generations=100,
                     population_size=100,
                     offspring_size=100, 
                     mutation_rate=0.9,
                     crossover_rate=0.1,
                     scoring='recall', 
                     cv=5,
                     subsample=1.0, 
                     n_jobs=1,
                     max_time_mins=None, 
                     max_eval_time_mins=5,
                     random_state=None, 
                     config_dict=None,
                     warm_start=True,
                     memory=None,
                     use_dask=False,
                     periodic_checkpoint_folder='models/',
                     early_stop=None,
                     verbosity=3,
                     disable_update_check=False)

# TPOT SETTINGS:
#   > generations = Number of iterations to the run pipeline optimization process.
#   > population_size = Number of individuals to retain in the genetic programming population every generation.
#   > offspring_size = Number of offspring to produce in each genetic programming generation. 
#   > mutation_rate = Mutation rate for the genetic programming algorithm in the range [0.0, 1.0].
#   > crossover_rate Crossover rate for the genetic programming algorithm in the range [0.0, 1.0].
#   > scoring = Function used to evaluate the quality of a given pipeline for the regression problem.
#               Note that it is recommended to use the neg version of mean squared error and related 
#               metrics so TPOT will minimize (instead of maximize) the metric. 
#   > cv = Cross-validation strategy used when evaluating pipelines. 
#   > subsample = Fraction of training samples that are used during the TPOT optimization process. 
#   > n_jobs = Number of processes to use in parallel for evaluating pipelines during the TPOT optimization process. 
#   > max_time_mins = How many minutes TPOT has to optimize the pipeline. 
#   > max_eval_time_mins = How many minutes TPOT has to evaluate a single pipeline. 
#   > random_state = The seed of the pseudo random number generator used in TPOT. 
#   > config_dict = A configuration dictionary for customizing the TPOT operators and parameters
#          Possible inputs are:
#               Python dictionary, TPOT will use your custom configuration,
#               string 'TPOT light', TPOT will use a built-in configuration with only fast models and preprocessors, or
#               string 'TPOT MDR', TPOT will use a built-in configuration specialized for genomic studies, or
#               string 'TPOT sparse': TPOT will use a configuration dictionary with a one-hot encoder and the operators normally included in TPOT that also support sparse matrices, or
#               None, TPOT will use the default TPOTRegressor configuration.
#   > warm_start = Flag indicating whether the TPOT instance will reuse the population from previous calls to fit(). 
#   > memory = If supplied, pipeline will cache each transformer after calling fit. 
#   > use_dask = Whether to use Dask-ML's pipeline optimiziations. 
#   > periodic_checkpoint_folder = f supplied, a folder in which TPOT will periodically save the best pipeline so far while optimizing.
#   > early_stop = How many generations TPOT checks whether there is no improvement in optimization process. 
#   > verbosity = How much information TPOT communicates while it's running. 
#   > disable_update_check = Flag indicating whether the TPOT version checker should be disabled.                      
#    Note: You can check the input parameters of regrtessor of classifier in 
#          https://epistasislab.github.io/tpot/api/

# Run traning
tpot.fit(X_train, y_train)

# Print last obtaned accuracy score after pipeline discovery
print(tpot.score(X_test, y_test))

30 operators have been imported by TPOT.


Skipped pipeline #11 due to time out. Continuing to the next pipeline.
Skipped pipeline #14 due to time out. Continuing to the next pipeline.
Skipped pipeline #20 due to time out. Continuing to the next pipeline.
Skipped pipeline #35 due to time out. Continuing to the next pipeline.
Skipped pipeline #40 due to time out. Continuing to the next pipeline.
Skipped pipeline #42 due to time out. Continuing to the next pipeline.
Skipped pipeline #46 due to time out. Continuing to the next pipeline.
Skipped pipeline #48 due to time out. Continuing to the next pipeline.
Skipped pipeline #59 due to time out. Continuing to the next pipeline.
Skipped pipeline #61 due to time out. Continuing to the next pipeline.
Skipped pipeline #65 due to time out. Continuing to the next pipeline.
Skipped pipeline #70 due to time out. Continuing to the next pipeline.
Skipped pipeline #72 due to time out. Continuing to the next pipeline.
Skipped pipeline #74 due to time out. Continuing to the next pipeline.
Skippe

_pre_test decorator: _random_mutation_operator: num_test=0 y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required..
_pre_test decorator: _random_mutation_operator: num_test=1 array must not contain infs or NaNs.
_pre_test decorator: _random_mutation_operator: num_test=0 y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required..
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 100.
_pre_test decorator: _random_mutation_operator: num_test=0 This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.
_pre_test decorator: _random_mutation_operator: num_test=1 y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required..
_pre_test decorator: _random_mutation_operator: num_test=2 Input X must be non

In [19]:
print('closing TPOT complete')

closing TPOT complete


In [20]:
# Create Models folder if not existent
model_folder = "models/"
if not os.path.isdir(model_folder):
    os.makedirs(model_folder)

# Define path to save TPOT optimized predictive model (.py)
model_filename = "optimized_tpot_pipeline.py"
model_path = model_folder + model_filename

# Save TPOT optimized predictive model (.py)
tpot.export(model_path)

In [21]:
y_preds = tpot.predict(X_test)

In [22]:
specs = 'AutoML Binary Classifier'

y_true = y_test

y_true_labels = y_true#np.argmax(y_true, axis=1)
y_pred_labels = y_preds#np.argmax(y_preds, axis=1) 
from sklearn.metrics import roc_auc_score,accuracy_score, f1_score, balanced_accuracy_score

print('AUC:', roc_auc_score(y_true_labels, y_pred_labels))

print('Accuracy:', accuracy_score(y_true_labels, y_pred_labels), '\n')
print('Balanced Accuracy:', balanced_accuracy_score(y_true_labels, y_pred_labels), '\n')
print('F1:', f1_score(y_true_labels,y_pred_labels, average='macro'),'\n')

class_names = ['0', '1']
confusion_plot(y_true_labels, y_pred_labels, 'Confusion Matrix: ' + specs, class_names)

AUC: 0.5
Accuracy: 0.001838612825302442 

Balanced Accuracy: 0.5 

F1: 0.0018352385321996503 

Confusion matrix, without normalization
[[     0 102063]
 [     0    188]]


/Users/kcollins/anaconda/envs/deeptoehold/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


All data confusion matrix
[[0. 1.]
 [0. 1.]]


/Users/kcollins/anaconda/envs/deeptoehold/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [23]:
# from TAs on piazza: use precision-recall, ROC can be misleading 
# modified code from sklearn documentation: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html
# and referenced: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature

average_precision = average_precision_score(y_true, y_preds)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

precision, recall, _ = precision_recall_curve(y_true_labels, y_preds) # look at positive class prob

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

Average precision-recall score: 0.00


Text(0.5, 1.0, '2-class Precision-Recall curve: AP=0.00')

In [14]:
tpot.scoring_function

'recall'

In [24]:
tpot.fitted_pipeline_

Pipeline(memory=None,
     steps=[('nystroem', Nystroem(coef0=None, degree=None, gamma=0.5, kernel='sigmoid',
     kernel_params=None, n_components=1, random_state=None)), ('rbfsampler', RBFSampler(gamma=0.15000000000000002, n_components=100, random_state=None)), ('gaussiannb', GaussianNB(priors=None, var_smoothing=1e-09))])

In [16]:
tpot.evaluated_individuals_

{'XGBClassifier(SelectPercentile(input_matrix, SelectPercentile__percentile=78), XGBClassifier__learning_rate=1.0, XGBClassifier__max_depth=6, XGBClassifier__min_child_weight=1, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.5)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': 0.16065520065520067},
 'GaussianNB(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.001, XGBClassifier__max_depth=6, XGBClassifier__min_child_weight=4, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.55))': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': 0.16245700245700245},
 'MultinomialNB(input_matrix, MultinomialNB__alpha=0.01, MultinomialNB__fit_prior=False)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('R